In [ ]:
import os, getpass

In [15]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
import openai

# Configura tu clave de API de OpenAI
openai.api_key = 'sk-proj-0XT3GGvdjgAQUVMOQrhBT3BlbkFJEvyFh5Mjlm8NbuNB507I'

url = "https://www.youtube.com/watch?v=5yyB6qmafuc" #5 minutos
save_dir = "docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser()
)
data = loader.load()


[youtube] Extracting URL: https://www.youtube.com/watch?v=5yyB6qmafuc
[youtube] 5yyB6qmafuc: Downloading webpage


[youtube] 5yyB6qmafuc: Downloading ios player API JSON
[youtube] 5yyB6qmafuc: Downloading m3u8 information
[info] 5yyB6qmafuc: Downloading 1 format(s): 140
[download] docs/youtube//Los límites físicos de la inteligencia artificial.m4a has already been downloaded
[download] 100% of    4.64MiB
[ExtractAudio] Not converting audio docs/youtube//Los límites físicos de la inteligencia artificial.m4a; file is already in target format m4a


AttributeError: module 'openai' has no attribute 'OpenAI'

In [ ]:
data

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 400, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)

## Cargando datos a Qrant

In [ ]:
url = "https://ac0d6ef8-b3e9-462f-b7ab-1d1385ad5d67.us-east4-0.gcp.cloud.qdrant.io" #Reemplaza por tu url
api_key = getpass.getpass("Ingresa tu API Key de Qdrant : ")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

embedding = OpenAIEmbeddings()

vectorstore = Qdrant.from_documents(
    docs,
    embedding,
    url = url,
    api_key = api_key,
    collection_name = "c_youtube",
    force_recreate = True
)

## Recuperando contexto de la colección en Qdrant

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

PROMPT = PromptTemplate(
    template = template, input_variables = ["context", "question"]
)

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever(),
    verbose = True,
    chain_type_kwargs = {"prompt": PROMPT, "verbose": True}
)

In [ ]:
qa.run("cuáles son los límites físicos de la IA")

In [ ]:
qa.run("de qué trata la ley de Moore")